<h1 align=center><font size = 6>Capstone project on Segmenting and Clustering Neighborhoods in Toronto_Part1</font></h1>


## Problem 1

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that - M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the - neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [33]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import requests
webUrl = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

print('Libraries imported.')

Libraries imported.


###### Scrape the List of postal codes of Canada

In [34]:
soup = BeautifulSoup(webUrl, 'xml')

In [35]:
table=soup.find('table')
#print(table)

In [36]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
data = pd.DataFrame(columns = column_names)

In [37]:
# Search all the postcode, borough, neighborhood 
for tr in table.find_all('tr'):
    rowData=[]
    for td in tr.find_all('td'):
        rowData.append(td.text.strip())
    if len(rowData)==3:
        data.loc[len(data)] = rowData
data.head(20)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


###### Data Cleaning
remove rows where Borough is 'Not assigned'

In [38]:
data=data[data['Borough']!='Not assigned']
data.head()


,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [39]:
data[data['Neighborhood']=='Not assigned']=data['Borough']
data.head()

ValueError: cannot set using a list-like indexer with a different length than the value

In [43]:
temp_data=data.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_data=temp_data.reset_index(drop=False)
temp_data.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [44]:
data_merge = pd.merge(data, temp_data, on='Postalcode')

In [45]:
data_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [46]:
data_merge.drop_duplicates(inplace=True)

In [47]:
data_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [48]:
data_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [49]:
data_merge.shape

(103, 3)